In [1]:
#!M
# !pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1
# %pip install --upgrade pip
# %pip install -r /home/jupyter/work/resources/riiidNew/requirements.txt --upgrade
# %pip install scipy  --ignore-installed scipy --upgrade
# %pip install pyarrow==0.17.1
# %pip install bayesian-optimization
print("ok")

ok


In [11]:
#!L
# if str(Path.home()) == "/root":  # DATASPHERE
import sys
import gc
from pathlib import Path
sys.path.append(str(Path.cwd()/'riiidNew'))
from pathlib import Path
import numpy as np
import pandas as pd
# import pyarrow
from collections import defaultdict
from catboost import CatBoostClassifier, Pool
from preprocess import preprocess_train_data
from catboost_bayesian_search import bayesian_catboost_search, bayesian_catboost_searchCV, param_adjust_dtypes
import config

In [4]:
# Error handling, ignore all
np.seterr(divide = 'ignore', invalid = 'ignore')


if str(Path.home()) == "/home/sergey":   # Home computer
    kaggle_path = Path.cwd()/"kaggle_tmp/"
    questions_df = pd.read_csv('/mnt/data30G/2020riiid/questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather", columns=dtypes.keys())
    train_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train_1e5.feather", columns=dtypes.keys())
    # test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather", columns=dtypes.keys())
    # train_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train.feather", columns=dtypes.keys())
    # test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid.feather", columns=dtypes.keys())

elif str(Path.home()) == "/root":   # DATASPHERE

    path = Path.cwd()/"riiidNew"/"data1"
    kaggle_path = Path.cwd()/"riiidNew"/"kaggle"
    questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_5e6.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())
    # train_df = pd.read_pickle(path/'cv2_train.pickle.zip').astype(config.dtypes, errors="ignore")
    # test_df = pd.read_pickle(path/"cv2_valid.pickle.zip").astype(config.dtypes, errors="ignore")

elif str(Path.home()) == "/home/riiid":   # Virtual machine ycloud

    path = Path.cwd()/"data"
    kaggle_path = Path.cwd()/"kaggle"
    questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    # train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_5e6.feather", columns=config.dtypes.keys())
    # test_df = pd.read_feather(path/"cv2_test_1e4.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"cv2_train_1e5.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())
#     train_df = train_df.iloc[:100]

#print(f"train_df shape = {train_df.shape}")

In [12]:
print(config.prior_params)

{'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': 'AUC:hints=skip_train~false', 'task_type': 'GPU', 'grow_policy': 'Lossguide', 'iterations': 5000, 'learning_rate': 0.1, 'random_seed': 0, 'bootstrap_type': 'Bayesian', 'l2_leaf_reg': 0.5, 'depth': 10, 'max_leaves': 10, 'border_count': 128, 'verbose': 250, 'od_type': 'Iter', 'od_wait': 50}


In [8]:
# # Obtain new train_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())
# # train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())

# # Preprocess for Bayesian CV
# train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
# # test_user_agg, test_content_agg, test_df = preprocess_train_data(test_df, questions_df, config.target, config.dtypes)
# print("Preprocess for CV ok")


# # Training and validating data
# train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
# val_set = Pool(test_df[config.features], label = test_df[config.target])

# print("optimizer_maxCV ..")
# optimizer_maxCV = bayesian_catboost_searchCV(
#     train_set,
#     prior_params=config.prior_params,
#     pds=config.pds, pds_dtypes=config.pds_dtypes,
#     init_points=5, n_iter=7, verbose=True,
# )

Preprocess for CV ok
optimizer_maxCV ..


In [13]:
# Obtain new train_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())
train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())


# Preprocess for model fitting
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
print("Preprocess for model fitting ok")

# Ratio is 6/24 = 25%
test_df = train_df.groupby('user_id').tail(6)
train_df.drop(test_df.index, inplace = True)

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
val_set = Pool(test_df[config.features], label = test_df[config.target], cat_features=config.cat_features)


optimizer_max = bayesian_catboost_search(
    train_set, val_set,
    prior_params=config.prior_params,
    pds=config.pds, pds_dtypes=config.pds_dtypes,
    init_points=5, n_iter=7, verbose=True,
)

print("optimizer_max ..")
print(optimizer_max)

Preprocess for model fitting ok
|   iter    |  target   | baggin... | border... |   depth   | l2_lea... | max_le... |
-------------------------------------------------------------------------------------
0:	learn: 0.7716994	test: 0.7187736	best: 0.7187736 (0)	total: 36.7ms	remaining: 3m 3s
bestTest = 0.7682361901
bestIteration = 70
Shrink model to first 71 iterations.
0:	learn: 0.7705329	test: 0.7186846	best: 0.7186846 (0)	total: 27.2ms	remaining: 2m 15s
bestTest = 0.7690668702
bestIteration = 191
Shrink model to first 192 iterations.
|  2        |  0.7691   |  3.336    |  159.4    |  23.16    |  481.8    |  63.68    |
0:	learn: 0.7626809	test: 0.7196122	best: 0.7196122 (0)	total: 17.2ms	remaining: 1m 26s
bestTest = 0.7708970308
bestIteration = 184
Shrink model to first 185 iterations.
|  3        |  0.7709   |  4.021    |  182.2    |  17.66    |  462.8    |  19.95    |
0:	learn: 0.7434526	test: 0.6910856	best: 0.6910856 (0)	total: 40.7ms	remaining: 3m 23s
bestTest = 0.7667251527
bestI

In [14]:
del train_df
gc.collect()

22

In [16]:
# Obtain new train_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())
train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())


# Preprocess for model fitting
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
print("Preprocess for model fitting ok")

# Ratio is 6/24 = 25%
test_df = train_df.groupby('user_id').tail(6)
train_df.drop(test_df.index, inplace = True)

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
val_set = Pool(test_df[config.features], label = test_df[config.target], cat_features=config.cat_features)


# val_set = Pool(valid_df[features], label = valid_df[target])
params = param_adjust_dtypes(config.prior_params,
                             config.pds_dtypes,
                             optimizer_max["params"])
params["learning_rate"] = 2e-2
params['iterations'] = 20000

print("params")
print(params)

model = CatBoostClassifier(**params)

# Fitting
model.fit(train_set, eval_set = val_set, use_best_model = True)
print(f"kaggle_path {kaggle_path}")

model.save_model(f"{kaggle_path/'catboost.model'}")

print("model.get_best_score()")
print(model.get_best_score())

Preprocess for model fitting ok
params
{'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': 'AUC:hints=skip_train~false', 'task_type': 'GPU', 'grow_policy': 'Lossguide', 'iterations': 20000, 'learning_rate': 0.02, 'random_seed': 0, 'bootstrap_type': 'Bayesian', 'l2_leaf_reg': 462.813199818672, 'depth': 17, 'max_leaves': 19, 'border_count': 182, 'verbose': 250, 'od_type': 'Iter', 'od_wait': 50, 'bagging_temperature': 4.021107678988524}
0:	learn: 0.7626809	test: 0.7196122	best: 0.7196122 (0)	total: 19.2ms	remaining: 6m 24s
250:	learn: 0.8077318	test: 0.7669528	best: 0.7669566 (249)	total: 4.19s	remaining: 5m 29s
500:	learn: 0.8144199	test: 0.7690084	best: 0.7690084 (500)	total: 8.51s	remaining: 5m 31s
bestTest = 0.7690390646
bestIteration = 506
Shrink model to first 507 iterations.
kaggle_path /home/riiid/riiidNew/kaggle
model.get_best_score()
{'learn': {'Logloss': 0.5184433600530507, 'AUC': 0.8157019317150116}, 'validation': {'Logloss': 0.56527326662497, 'AUC': 0.76903906

In [17]:
from pathlib import Path
Path.home()

PosixPath('/home/riiid')

# Inference

In [18]:
#!M
import pickle

user_sum_dict = train_user_agg['sum'].astype('int32').to_dict(defaultdict(int))
user_count_dict = train_user_agg['count'].astype('int32').to_dict(defaultdict(int))
content_sum_dict = train_content_agg['sum'].astype('int32').to_dict(defaultdict(int))
content_count_dict = train_content_agg['count'].astype('int32').to_dict(defaultdict(int))

for filename, dic in zip(["user_sum_dict", "user_count_dict", "content_sum_dict", "content_count_dict"],
                         [user_sum_dict, user_count_dict, content_sum_dict, content_count_dict]):
    with open(f'{kaggle_path}/{filename}.pickle', 'wb') as handle:
        pickle.dump(dic, handle)

In [ ]:
train_df.head()
del train_df
gc.collect()

In [ ]:
%%time
validaten_flg = True
if validaten_flg:
    from emulator import Iter_Valid
    iter_test = Iter_Valid(test_df,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict

In [ ]:
%%time

prior_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    if prior_test_df is not None:
        prior_test_df[config.target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df[config.target] != -1].reset_index(drop = True)

        user_ids = prior_test_df['user_id'].values
        content_ids = prior_test_df['content_id'].values
        targets = prior_test_df[config.target].values

        # for user_id, content_id, answered_correctly in prior_test_df[["user_id", "content_id", target]].values:
        for user_id, content_id, answered_correctly in zip(user_ids, content_ids, targets):
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
            content_sum_dict[content_id] += answered_correctly
            content_count_dict[content_id] += 1

    prior_test_df = test_df.copy()

    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)
    test_df.drop(labels="part", axis=1, inplace=True)
    test_df.content_id = test_df.content_id.astype(int)

    test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(False).astype('bool')
    user_sum = np.zeros(len(test_df), dtype = np.int32)
    user_count = np.zeros(len(test_df), dtype = np.int32)
    content_sum = np.zeros(len(test_df), dtype = np.int32)
    content_count = np.zeros(len(test_df), dtype = np.int32)

    for i, (user_id, content_id) in enumerate(zip(test_df['user_id'].values, test_df['content_id'].values)):
        user_sum[i] = user_sum_dict[user_id]
        user_count[i] = user_count_dict[user_id]
        content_sum[i] = content_sum_dict[content_id]
        content_count[i] = content_count_dict[content_id]

    test_df['user_correctness'] = user_sum / user_count
    test_df['content_count'] = content_count
    test_df['content_id'] = content_sum / content_count
    test_df[config.target] = model.predict_proba(test_df[config.features])[:,1]
    set_predict(test_df[['row_id', config.target]])